In [1]:
import os
import tensorflow as tf
from keras import models
from keras import layers
from keras import Sequential
from datetime import datetime
from PIL import Image
import numpy as np
import keras.backend as K
import matplotlib.pyplot as plt
import mlflow
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt
from keras.models import Model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score

In [2]:

# Define the path to your dataset folder and other necessary variables
dataset_name = 'dataset_2_final'  # Replace with your dataset folder path
batch_size = 32
num_classes = 3  # Replace with the number of classes in your dataset
epochs = 10

In [3]:
# Load the dataset
image_size = (300, 300)

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_name,
    validation_split=0.2,
    subset="training",
    label_mode='int',
    labels='inferred',
    seed=1336,
    image_size=image_size,
    interpolation='bilinear',
    batch_size=batch_size,
    shuffle=True,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_name,
    validation_split=0.2,
    subset="validation",
    label_mode='int',
    labels='inferred',
    seed=1336,
    image_size=image_size,
    interpolation='bilinear',
    batch_size=batch_size,
    shuffle=True,
)


Found 494 files belonging to 7 classes.
Using 396 files for training.
Found 494 files belonging to 7 classes.
Using 98 files for validation.


In [5]:
# Custom data generator function that crops images into four pieces
def custom_data_generator(ds, crop_size, batch_size):
    for images, labels in ds:
        batch_crops = []
        for image in images:
            crops = []
            img_height, img_width, _ = image.shape

            # Crop the image into four pieces
            for i in range(2):
                for j in range(2):
                    start_x = i * crop_size[0]
                    start_y = j * crop_size[1]
                    end_x = start_x + crop_size[0]
                    end_y = start_y + crop_size[1]
                    crop = image[start_y:end_y, start_x:end_x, :]
                    crops.append(crop)

            batch_crops.extend(crops)

        # Stack the cropped images to form the batch
        batch_crops = np.stack(batch_crops)
        yield batch_crops, np.tile(labels, (4, 1))

In [6]:

# Initialize the EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=image_size + (3,))
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:

# Print the model summary
model.summary()

# Train the model using the custom data generator
train_gen = custom_data_generator(train_ds, crop_size=(150, 150), batch_size=batch_size)
val_gen = custom_data_generator(val_ds, crop_size=(150, 150), batch_size=batch_size)

steps_per_epoch = len(train_ds) * 4 // batch_size  # Four crops per image
validation_steps = len(val_ds) * 4 // batch_size  # Four crops per image

model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=epochs, validation_data=val_gen, validation_steps=validation_steps)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 300, 300, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 300, 300, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 301, 301, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

ResourceExhaustedError:  OOM when allocating tensor with shape[128,144,75,75] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/block2a_expand_bn/FusedBatchNormV3 (defined at C:\Users\diosdadj\AppData\Local\Temp\ipykernel_35700\2369071688.py:11) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_26250]

Function call stack:
train_function
